In [31]:
source("ancom_custom.R")
library(tidyverse)
library(firatheme)

In [32]:
feature_table <- read.table("../data/hmp_feature.txt", sep="\t", header=1, row.names=1)
meta_data <- read.table("../data/hmp_meta.txt", sep="\t", header=1)

In [33]:
feature_table = feature_table; meta_data = meta_data; sample_var = "ID"; group_var = "oxygen_availability";
out_cut = 0.05; zero_cut = 0.9; lib_cut = 0; neg_lb = FALSE
prepro = feature_table_pre_process(feature_table, meta_data, sample_var, group_var, 
                                   out_cut, zero_cut, lib_cut, neg_lb)
feature_table = prepro$feature_table # Preprocessed feature table
meta_data = prepro$meta_data # Preprocessed metadata
struc_zero = prepro$structure_zeros # Structural zero info

In [34]:
# ANCOM without covariate
main_var = "oxygen_availability"; p_adj_method = "BH"; alpha = 0.05
adj_formula = NULL; rand_formula = NULL
t_start = Sys.time()
res = ANCOM(feature_table, meta_data, struc_zero, main_var, p_adj_method, 
            alpha, adj_formula, rand_formula)
t_end = Sys.time()
t_run_w = t_end - t_start

[1] "Cutoff value for W:  628.1"
[1] "detected_0.9:  628.2"
[1] "detected_0.8:  558.4"
[1] "detected_0.7:  488.6"
[1] "detected_0.6:  418.8"


In [35]:
print(t_run_w)

Time difference of 11.90709 mins


In [36]:
as.character(drop_na(res$out[res$out$detected_0.9,])$taxa_id)

[1] "Actinomyces"           "Corynebacterium"       "Rothia"               
 [4] "Propionibacterium"     "Bifidobacterium"       "Bacteroides"          
 [7] "Barnesiella"           "Butyricimonas"         "Odoribacter"          
[10] "Parabacteroides"       "Alistipes"             "Gemella"              
[13] "Staphylococcus"        "Streptococcus"         "Anaerovorax"          
[16] "Blautia"               "Anaerococcus"          "Finegoldia"           
[19] "Peptoniphilus"         "Anaerostipes"          "Coprococcus"          
[22] "Dorea"                 "Pseudobutyrivibrio"    "Roseburia"            
[25] "Acetivibrio"           "Anaerotruncus"         "Butyricicoccus"       
[28] "Faecalibacterium"      "Oscillibacter"         "Ruminococcus"         
[31] "Sporobacter"           "Subdoligranulum"       "Dialister"            
[34] "Phascolarctobacterium" "Coprobacillus"         "Holdemania"           
[37] "Turicibacter"          "Fusobacterium"         "Parasutterella"       
[40] "Sutterella"            "Desulfovibrio"         "Acinetobacter"        
[43] "Pseudomonas"           "Akkermansia"

In [37]:
# ANCOM with a covariate
main_var = "oxygen_availability"; p_adj_method = "BH"; alpha = 0.05
adj_formula = "body_site"; rand_formula = NULL
t_start = Sys.time()
res2 = ANCOM(feature_table, meta_data, struc_zero, main_var, p_adj_method, 
            alpha, adj_formula, rand_formula)
t_end = Sys.time()
t_run_wo = t_end - t_start

[1] "Cutoff value for W:  558.2"
[1] "detected_0.9:  628.2"
[1] "detected_0.8:  558.4"
[1] "detected_0.7:  488.6"
[1] "detected_0.6:  418.8"


In [38]:
print(t_run_wo)

Time difference of 5.794084 mins


In [39]:
as.character(drop_na(res2$out[res2$out$detected_0.9,])$taxa_id)

[1] "Corynebacterium"       "Propionibacterium"     "Bacteroides"          
 [4] "Barnesiella"           "Butyricimonas"         "Odoribacter"          
 [7] "Parabacteroides"       "Paraprevotella"        "Prevotella"           
[10] "Alistipes"             "Staphylococcus"        "Anaerovorax"          
[13] "Blautia"               "Anaerostipes"          "Coprococcus"          
[16] "Dorea"                 "Roseburia"             "Acetivibrio"          
[19] "Anaerotruncus"         "Butyricicoccus"        "Faecalibacterium"     
[22] "Oscillibacter"         "Ruminococcus"          "Sporobacter"          
[25] "Subdoligranulum"       "Acidaminococcus"       "Dialister"            
[28] "Phascolarctobacterium" "Coprobacillus"         "Parasutterella"       
[31] "Akkermansia"

In [40]:
# Number of taxa except structural zeros
n_taxa = ifelse(is.null(struc_zero), nrow(feature_table), sum(apply(struc_zero, 1, sum) == 0))
# Cutoff values for declaring differentially abundant taxa
cut_off = c(0.9 * (n_taxa -1), 0.8 * (n_taxa -1), 0.7 * (n_taxa -1), 0.6 * (n_taxa -1), res$autoW)
names(cut_off) = c("detected_0.9", "detected_0.8", "detected_0.7", "detected_0.6","auto_W")

# Annotation data
dat_ann = data.frame(x = min(res$fig$data$x), y = cut_off["detected_0.7"], label = "W[0.7]")
dat_ann2 = data.frame(x = min(res$fig$data$x), y = cut_off["auto_W"], label = "W(auto)")

fig = ggplot(data = res$fig$data) + aes(x = x, y = y, color=zero_ind) + 
  geom_point(size=4, alpha=.5)+
  labs(x = "CLR mean difference", y = "W statistic") +
  scale_color_fira(name = "Structural zero", drop = FALSE) + 
  theme_bw() +
  geom_hline(yintercept = cut_off["detected_0.7"], linetype = "dashed") + 
  geom_text(data = dat_ann, aes(x = x, y = y, label = label), 
            size = 4, vjust = -0.5, hjust = 0, color = firaPalette()[2], parse = TRUE)+
  geom_hline(yintercept = cut_off["auto_W"], linetype = "dashed") + 
  geom_text(data = dat_ann2, aes(x = x, y = y, label = label), 
            size = 4, vjust = -0.5, hjust = 0, color = firaPalette()[3], parse = TRUE)+
  theme(plot.title = element_text(hjust = 0.5), legend.position = "top") + ggtitle("Without covariate")

In [41]:
# Number of taxa except structural zeros
n_taxa = ifelse(is.null(struc_zero), nrow(feature_table), sum(apply(struc_zero, 1, sum) == 0))
# Cutoff values for declaring differentially abundant taxa
cut_off = c(0.9 * (n_taxa -1), 0.8 * (n_taxa -1), 0.7 * (n_taxa -1), 0.6 * (n_taxa -1), res2$autoW)
names(cut_off) = c("detected_0.9", "detected_0.8", "detected_0.7", "detected_0.6","auto_W")

# Annotation data
dat_ann = data.frame(x = min(res2$fig$data$x), y = cut_off["detected_0.7"], label = "W[0.7]")
dat_ann2 = data.frame(x = min(res2$fig$data$x), y = cut_off["auto_W"], label = "W(auto)")

fig2 = ggplot(data = res2$fig$data) + aes(x = x, y = y, color=zero_ind) + 
  geom_point(size=4, alpha=.5)+
  labs(x = "CLR mean difference", y = "W statistic") +
  scale_color_fira(name = "Structural zero", drop = FALSE) + 
  theme_bw() +
  geom_hline(yintercept = cut_off["detected_0.7"], linetype = "dashed") + 
  geom_text(data = dat_ann, aes(x = x, y = y, label = label), 
            size = 4, vjust = -0.5, hjust = 0, color = firaPalette()[2], parse = TRUE)+
  geom_hline(yintercept = cut_off["auto_W"], linetype = "dashed") + 
  geom_text(data = dat_ann2, aes(x = x, y = y, label = label), 
            size = 4, vjust = -0.5, hjust = 0, color = firaPalette()[3], parse = TRUE)+
  theme(plot.title = element_text(hjust = 0.5), legend.position = "top") + ggtitle("With a covariate")

In [42]:
library(cowplot)
png("../volcano_plot.png", res=200, width=9, height=5, unit="in")
plot_grid(fig, fig2, nrow=1)
dev.off()

Warning message:
“Removed 505 rows containing missing values (geom_point).”


pdf 
  2

In [43]:
# read LEfSe results
lefse.all <- read.table("../data/lefse.all.res", sep="\t")
lefse.all <- lefse.all[lefse.all$V5!="-",]
lefse.all$genus <- sapply(strsplit(as.character(lefse.all$V1), "[.]"), "[", 6)
lefse.all <- lefse.all[!is.na(lefse.all$genus),]

In [44]:
lefse.sub <- read.table("../data/lefse.sub.res", sep="\t")
lefse.sub <- lefse.sub[!is.na(lefse.sub$V4),]
lefse.sub$genus <- sapply(strsplit(as.character(lefse.sub$V1), "[.]"), "[", 6)
lefse.sub <- lefse.sub[!is.na(lefse.sub$genus),]

In [45]:
# load library for plotting
library(ComplexHeatmap)

In [46]:
# Make set for upset plot

genus_ovl <- list(lefse_with_subclass = lefse.all$genus,
     lefse_without_subclass = lefse.all$genus,
     ancom_with_covariate = as.character(drop_na(res2$out[res2$out$detected_0.9,])$taxa_id),
     ancom_without_covariate = as.character(drop_na(res$out[res$out$detected_0.9,])$taxa_id)
     )

m = make_comb_mat(genus_ovl)
cs = comb_size(m)
ss = set_size(m)

In [47]:
# Show intersection size above column: https://support.bioconductor.org/p/118557/
# modified to show set size at right side of right annotatiion

ht = UpSet(m,
          top_annotation = upset_top_annotation(m, gp = gpar(fill = firaPalette()[1]), ylim = c(0, 1.1*max(cs))),
          right_annotation = upset_right_annotation(m, gp = gpar(fill = firaPalette()[5]), xlim = c(0, 1.1*max(ss))),
          comb_col = firaPalette()[1], lwd=3,
          pt_size = unit(7, "mm"))

nc = ncol(m)
nr = nrow(m)

# Output

png("../upset_plot.png", res=150, width=10, height=5, unit="in")
ht = draw(ht, height=unit(3, "in"),
         padding = unit(c(0.2, 0.1, 0.2, 0.9), "cm"))
co = column_order(ht)
ro = row_order(ht)

decorate_annotation("Intersection\nsize", {
    grid.text(cs[co], 
        x = 1:nc, 
        y = unit(cs[co], "native") + unit(1, "mm"), 
        gp = gpar(fontsize = 15), 
        just = "bottom",
        default.units = "native")
})

decorate_annotation("Set size", {
    grid.text(ss[ro], 
        y = nr:1, 
        x = unit(ss[ro], "native") + unit(4, "mm"), 
        gp = gpar(fontsize = 15), 
        just = "bottom",
        default.units = "native")
})
dev.off()

pdf 
  2